# 範例 : (Kaggle)房價預測精簡版 
https://www.kaggle.com/c/house-prices-advanced-regression-techniques
***
- 以下是房價預測的精簡版範例
- 使用最小量的特徵工程以及線性回歸模型做預測, 最後輸出可以在Kaggle提交的預測檔

# [教學目標]
- 以下程式碼雖然與 Day16 類似, 但是主要重點在於特徵工程的使用, 後續的課程當中會教導同學如何對這塊作調整

# [範例重點]
- 精簡後的特徵工程 - 包含補缺失值(fillna). 標籤編碼(LabelEncoder).  
最小最大化(MinMaxScaler) 如何使用在同一個程式區塊中 (In[3])   

In [12]:
# 載入基本套件
import pandas as pd
import numpy as np

# 載入標籤編碼與最小最大化, 以便做最小的特徵工程
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# 讀取訓練與測試資料
data_path = '../data/'
df_train = pd.read_csv(data_path + 'house_train.csv.gz')
df_test = pd.read_csv(data_path + 'house_test.csv.gz')
print(df_train.shape)

(1460, 81)


In [13]:
# 訓練資料需要 train_X, train_Y / 預測輸出需要 ids(識別每個預測值), test_X
# 在此先抽離出 train_Y 與 ids, 而先將 train_X, test_X 該有的資料合併成 df, 先作特徵工程
train_Y = np.log1p(df_train['SalePrice'])
ids = df_test['Id']
df_train = df_train.drop(['Id', 'SalePrice'] , axis=1)
df_test = df_test.drop(['Id'] , axis=1)
df = pd.concat([df_train,df_test])
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
5,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal
6,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
7,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal
8,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml
9,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,1,2008,WD,Normal


In [14]:
# 特徵工程-簡化版 : 全部空值先補-1, 所有類別欄位先做 LabelEncoder, 然後再與數字欄位做 MinMaxScaler
# 這邊使用 LabelEncoder 只是先將類別欄位用統一方式轉成數值以便輸入模型, 當然部分欄位做 One-Hot可能會更好, 只是先使用最簡單版本作為範例
LEncoder = LabelEncoder()
# 除上述之外, 還要把標籤編碼與數值欄位一起做最大最小化, 這麼做雖然有些暴力, 卻可以最簡單的平衡特徵間影響力
MMEncoder = MinMaxScaler()
for c in df.columns:
    if df[c].dtype == 'object': # 如果是文字型 / 類別型欄位, 就先補缺 'None' 後, 再做標籤編碼
        df[c] = df[c].fillna('None')
        df[c] = LEncoder.fit_transform(df[c])
        print(df[c])
    else: # 其他狀況(本例其他都是數值), 就補缺 -1
        df[c] = df[c].fillna(-1)
        print(df[c])
    # 最後, 將標籤編碼與數值欄位一起最大最小化, 因為需要是一維陣列, 所以這邊切出來後用 reshape 降維
    df[c] = MMEncoder.fit_transform(df[c].values.reshape(-1, 1))
df

0        60
1        20
2        60
3        70
4        60
5        50
6        20
7        60
8        50
9       190
10       20
11       60
12       20
13       20
14       20
15       45
16       20
17       90
18       20
19       20
20       60
21       45
22       20
23      120
24       20
25       20
26       20
27       20
28       20
29       30
       ... 
1429     30
1430     50
1431     30
1432    190
1433     50
1434    120
1435    120
1436     20
1437     90
1438     20
1439     80
1440     20
1441     20
1442     20
1443     20
1444     20
1445     90
1446    160
1447     20
1448     90
1449    180
1450    160
1451     20
1452    160
1453    160
1454    160
1455    160
1456     20
1457     85
1458     60
Name: MSSubClass, Length: 2919, dtype: int64
0       4
1       4
2       4
3       4
4       4
5       4
6       4
7       4
8       5
9       4
10      4
11      4
12      4
13      4
14      4
15      5
16      4
17      4
18      4
19      4
20      4
21      5
22 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to floa


0       0
1       0
2       0
3       2
4       0
5       0
6       0
7       0
8       2
9       0
10      0
11      0
12      4
13      0
14      4
15      0
16      0
17      4
18      0
19      4
20      0
21      0
22      0
23      4
24      0
25      0
26      4
27      0
28      4
29      1
       ..
1429    2
1430    2
1431    4
1432    4
1433    1
1434    0
1435    0
1436    4
1437    4
1438    0
1439    4
1440    4
1441    0
1442    0
1443    0
1444    4
1445    2
1446    4
1447    0
1448    4
1449    4
1450    4
1451    1
1452    4
1453    4
1454    2
1455    4
1456    0
1457    4
1458    0
Name: HeatingQC, Length: 2919, dtype: int32
0       1
1       1
2       1
3       1
4       1
5       1
6       1
7       1
8       1
9       1
10      1
11      1
12      1
13      1
14      1
15      1
16      1
17      1
18      1
19      1
20      1
21      1
22      1
23      1
24      1
25      1
26      1
27      1
28      1
29      0
       ..
1429    1
1430    1
1431    0
1432 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to floa

0       4
1       4
2       4
3       0
4       4
5       4
6       4
7       4
8       0
9       4
10      4
11      5
12      4
13      5
14      4
15      4
16      4
17      4
18      4
19      0
20      5
21      4
22      4
23      4
24      4
25      4
26      4
27      4
28      4
29      4
       ..
1429    4
1430    4
1431    0
1432    0
1433    4
1434    5
1435    4
1436    4
1437    4
1438    4
1439    4
1440    2
1441    4
1442    5
1443    5
1444    4
1445    4
1446    4
1447    4
1448    4
1449    4
1450    4
1451    4
1452    0
1453    4
1454    4
1455    0
1456    0
1457    4
1458    4
Name: SaleCondition, Length: 2919, dtype: int32


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.235294,0.8,0.210191,0.033420,1.0,0.5,1.000000,1.000000,0.0,1.00,...,0.000000,0.0,1.0,1.00,0.25,0.000000,0.090909,0.50,1.000000,0.8
1,0.000000,0.8,0.257962,0.038795,1.0,0.5,1.000000,1.000000,0.0,0.50,...,0.000000,0.0,1.0,1.00,0.25,0.000000,0.363636,0.25,1.000000,0.8
2,0.235294,0.8,0.219745,0.046507,1.0,0.5,0.000000,1.000000,0.0,1.00,...,0.000000,0.0,1.0,1.00,0.25,0.000000,0.727273,0.50,1.000000,0.8
3,0.294118,0.8,0.194268,0.038561,1.0,0.5,0.000000,1.000000,0.0,0.00,...,0.000000,0.0,1.0,1.00,0.25,0.000000,0.090909,0.00,1.000000,0.0
4,0.235294,0.8,0.270701,0.060576,1.0,0.5,0.000000,1.000000,0.0,0.50,...,0.000000,0.0,1.0,1.00,0.25,0.000000,1.000000,0.50,1.000000,0.8
5,0.176471,0.8,0.273885,0.059899,1.0,0.5,0.000000,1.000000,0.0,1.00,...,0.000000,0.0,1.0,0.50,0.75,0.041176,0.818182,0.75,1.000000,0.8
6,0.000000,0.8,0.242038,0.041057,1.0,0.5,1.000000,1.000000,0.0,1.00,...,0.000000,0.0,1.0,1.00,0.25,0.000000,0.636364,0.25,1.000000,0.8
7,0.235294,0.8,0.000000,0.042450,1.0,0.5,0.000000,1.000000,0.0,0.00,...,0.000000,0.0,1.0,1.00,0.75,0.020588,0.909091,0.75,1.000000,0.8
8,0.176471,1.0,0.165605,0.022529,1.0,0.5,1.000000,1.000000,0.0,1.00,...,0.000000,0.0,1.0,1.00,0.25,0.000000,0.272727,0.50,1.000000,0.0
9,1.000000,0.8,0.162420,0.028605,1.0,0.5,1.000000,1.000000,0.0,0.00,...,0.000000,0.0,1.0,1.00,0.25,0.000000,0.000000,0.50,1.000000,0.8


In [15]:
# 將前述轉換完畢資料 df , 重新切成 train_X, test_X, 因為不論何種特徵工程, 都需要對 train / test 做同樣處理
# 常見並簡便的方式就是 - 先將 train / test 接起來, 做完後再拆開, 不然過程當中往往需要將特徵工程部分寫兩次, 麻煩且容易遺漏
# 在較複雜的特徵工程中尤其如此, 若實務上如果碰到 train 與 test 需要分階段進行, 則通常會另外寫成函數處理
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]

# 使用線性迴歸模型 : train_X, train_Y 訓練模型, 並對 test_X 做出預測結果 pred
from sklearn.linear_model import LinearRegression
estimator = LinearRegression()
estimator.fit(train_X, train_Y)
pred = estimator.predict(test_X)

In [17]:
# 將輸出結果 pred 與前面留下的 ID(ids) 合併, 輸出成檔案
# 可以下載並點開 house_baseline.csv 查看結果, 以便了解預測結果的輸出格式
# 本範例所與作業所輸出的 csv 檔, 均可用於本題的 Kaggle 答案上傳, 可以試著上傳來熟悉 Kaggle 的介面操作
pred = np.expm1(pred)
sub = pd.DataFrame({'Id': ids, 'SalePrice': pred})
sub.to_csv('house_baseline.csv', index=False) 

# 作業1
* 下列A~E五個程式區塊中，哪一塊是特徵工程?

# 作業2
* 對照程式區塊 B 與 C 的結果，請問那些欄位屬於"類別型欄位"? (回答欄位英文名稱即可) 

# 作業3
* 續上題，請問哪個欄位是"目標值"?